In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import seaborn as sns
import datetime
# plt.rcParams['axes.facecolor'] = 'white'

import warnings
warnings.filterwarnings('ignore')

In [2]:
root_path = '~/deepdow_poc'
raw_data = root_path + '/raw_data/'

In [3]:
"""
We will load the dataset for each assets, filter only the adj close column to use as our feature and concat the assets into same dataframe
"""

def load_prepare_data(assets_list, suffix, col=['Adj Close']): #, 'Volume']):
    assets_dict = {}
    temp_list = []
    
    if suffix != None:
        for asset in assets_list:
            df_temp = pd.read_csv(raw_data + asset + suffix +'.csv')
            df_temp['Date'] = pd.to_datetime(df_temp['Date'])
            df_temp.set_index('Date', inplace=True)
            df_temp = pd.DataFrame(df_temp[col])

            assets_dict[asset] = df_temp
            temp_list.append(df_temp)
    else:
        for asset in assets_list:
            df_temp = pd.read_csv(raw_data + asset +'.csv')
            df_temp['Date'] = pd.to_datetime(df_temp['Date'])
            df_temp.set_index('Date', inplace=True)
            df_temp = pd.DataFrame(df_temp[col])

            assets_dict[asset] = df_temp
            temp_list.append(df_temp)
            
    df = pd.concat(temp_list, keys=assets_list, axis=1)
    return assets_dict, df


ASSETS = ['AAPL', 'TSLA', 'SHOP', 'HLFNX','VGT']
_,df_1 = load_prepare_data(ASSETS, '_train')
_,df_2 = load_prepare_data(ASSETS, '_test')
df = pd.concat([df_1, df_2], axis=0)

In [4]:
"""
Convert the data of the asset price to daily returns
"""

def to_return(data, col, log=True):
    df = data.copy()
    for asset in df.columns.levels[0]:
        if log:
            df[asset, 'Adj Close'] = df[asset, 'Adj Close'].pct_change().apply(lambda x: np.log(1+x))
        else:
            df[asset, 'Adj Close'] = df[asset, 'Adj Close'].pct_change()
            
    return df.iloc[1:]

df_1 = to_return(df_1, 'Adj Close')
df_2 = to_return(df_2, 'Adj Close')
df = to_return(df, 'Adj Close')

In [5]:
df

,AAPL,TSLA,SHOP,HLFNX,VGT
,Adj Close,Adj Close,Adj Close,Adj Close,Adj Close
Date,,,,,
2019-01-03,-0.104924,-0.031978,-0.058433,-0.020795,-0.050685
2019-01-04,0.041803,0.056094,0.061771,0.035463,0.042565
2019-01-07,-0.002228,0.052935,0.044830,0.006221,0.011111
2019-01-08,0.018884,0.001164,0.007246,0.003096,0.009570
2019-01-09,0.016839,0.009438,0.021629,0.008209,0.012405
...,...,...,...,...,...
2020-12-23,-0.007000,0.008769,-0.063956,0.002863,-0.009144
2020-12-24,0.007683,0.024150,0.022745,0.004635,0.006625


In [6]:
"""
Establish constants
1. Rebalance freq = BMS first business day of month
2. Lookback = number of historical data in unit of days that 
   deep learning use to learn to predict weight for the horizon
3. Horizon = days that user hold before rebalance. Corresponds to the Rebalance freq. 
    20 days = n_business_day in 1 month
"""

LOOKBACK = 40
HORIZON = 20
WINDOW = 250
N_SAMPLES = WINDOW - LOOKBACK - HORIZON
N_ASSETS = len(df.columns.levels[0])
N_FEATURES = LOOKBACK * N_ASSETS

"""
first day of the month in the historical data
go through each element in list, if month changes, that is the first day of month, assuming time is ordered in the list
"""
def get_first_date(datelist):
    rebal_dates = []
    for index,d in enumerate(df.index):
        if index==0:
            rebal_dates.append(d)
            current_month = d.month
        else:
            if d.month != current_month:
                rebal_dates.append(d)
                current_month = d.month
            else:
                continue
    return rebal_dates

REBAL_DATES = get_first_date(df.index)


In [7]:
len(df.columns.levels[0])

5

In [8]:
"""
Train and optimize.
Training and optimize will be done to 1 rolling window to predict the weight to use for the horizon.
Rolling window will start from T(rebalance date) and go backwards
"""

# first will define the loss functions
import tensorflow as tf

def portfolio_returns(weights, y):
    """
    Input
    --------
    weights = the predicted values output from neural network. (n_samples, n_asset)
    y = test data (n_samples, horizon, n_asset) use for loss calculation
    """
    # exp to reverse log transform
    y = tf.math.exp(y) - 1
    n_samples = tf.shape(y)[0]
    horizon = tf.shape(y)[1]
    n_assets = tf.shape(y)[2]
    
    """ 
    reshape the weight into (n_samples, horizon, n_asset) dimension
    first we have (n_samples, n_assets) weight output from neural network output
    we want to repeat the vector horizon amount of time
    """
    weights_ = tf.tile(tf.reshape(weights, [n_samples, 1, n_assets]), (1, horizon, 1))
    
    """
    rebalance : now will always be false (same for all time steps in the horizon)
        If True, each timestep the weights are adjusted to be equal to be equal to the original ones. Note that
        this assumes that we tinker with the portfolio. If False, the portfolio evolves untouched.
    """
    rebalance=False
    
    if not rebalance:
        weights_unscaled = tf.math.cumprod((1 + y),1)[:, :-1, :] * weights_[:, 1:, :]
        
        output_list = []
        output_list.append(weights_[:, 0, :])
        
        remaining = weights_unscaled / tf.math.reduce_sum(weights_unscaled, 2, keepdims=True)
        rows = remaining.get_shape()
        for row in range(rows[1]):
            output_list.append(remaining[:, row,:])
        
        new_weights = tf.stack(output_list, axis=1)

    out = tf.math.reduce_sum((y * new_weights),-1)
    
    # shape (n_samples, horizon) representing per timestep portfolio returns
    return out

def MeanReturns(y, weights):#, rebalance=False):
    """Negative mean returns"""
    prets = portfolio_returns(weights, y[:, 0, ...])#, rebalance=rebalance)
    
    """Tensor of shape `(n_samples,)` representing the per sample negative mean returns."""
    # neg_mean = tf.math.exp(prets)
    mean = tf.math.reduce_mean(prets, 1)
    neg_mean = mean * -1
    return neg_mean

def SharpeRatio(y, weights, rf=0.01, eps=1e-4):
    """Negative Sharpe ratio"""
    prets = portfolio_returns(weights, y[:, 0, ...])#, rebalance=rebalance)
    
    """Tensor of shape `(n_samples,)` representing the per sample negative mean returns."""
    sharpe = ((tf.math.reduce_mean(prets, 1) - rf)/(tf.math.reduce_std(prets, 1) + eps))
    neg_sharpe = sharpe * -1
    return neg_sharpe

def LargestWeight(y, weights):
    """Penalize low diversity, or over powering weight in 1 asset, lower = better"""
    return tf.reduce_max(weights, 1)[0]

def StandardDeviation(y, weights):
    """Returns std dev of portfolio return of the horizon, lower = better"""
    prets = portfolio_returns(weights, y[:, 0, ...])
    return tf.math.reduce_std(prets, 1)

def make_my_loss(alpha):
    def my_loss(y, weights):
        """Combination of different loss functions for the final loss calculation"""
        return (alpha[0] * MeanReturns(y, weights)
                + alpha[1] * SharpeRatio(y, weights)
                + alpha[2] * StandardDeviation(y, weights)
               )
    return my_loss


In [9]:
"""
Define the model archtecture
-----------------------------
This is a simple feedfoward model that the input is the flattened array of (n_samples, n_features)
n_features = n_assets * horizon

The output is softmax of dimension n_assets. Softmax --> sum of all weight = 1
"""
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Softmax
from tensorflow.keras.activations import softmax

def build_model(n_assets, input_shape, dropout):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = Dropout(dropout)(x)
    weights = Dense(n_assets, activation="linear")(x)
    outputs = Softmax()(weights)

    return keras.Model(inputs, outputs)

In [ ]:
"""
Train and predict
1. Get the rolling window data from of T-250
2. scale and split data to prepare for neural network input, Xtrain, Xtest, ytrain, ytest, Xpred
3. Train the model with Xtrain,Xtest,ytrain,ytest
4. Predict weight using Xpred
5. Save weight for that T
6. Repeat 1 - 5 for every T(rebal date)
"""
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=50, verbose=2, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)
N_EPOCHS = 1000
BATCH_SIZE = 32

# we will start the prediction from year 2020
pred_dates = [d for d in REBAL_DATES if d.year == 2020]

def get_window(df,pred_date):
    temp = df.copy()
    pred_date_ind = np.where(temp.index == pred_date)[0][0]
    window = temp.iloc[((pred_date_ind+1)-WINDOW):(pred_date_ind+1)]
    return window

def create_input(df):
    data = df.copy()
    scaler = MinMaxScaler(feature_range=(0,1))
    transformer = scaler.fit(data)
    scaled_data = pd.DataFrame(transformer.transform(data), columns=data.columns)
    test_data = scaled_data.iloc[:100]
    train_data = scaled_data.iloc[100:]
    
    n_test_sample = 100 - LOOKBACK - HORIZON
    n_train_sample = 100 - LOOKBACK - HORIZON
    
    assets = scaled_data.columns.levels[0].tolist()
    channels = scaled_data.columns.levels[1].tolist()
    
    n_channels = len(data.columns.levels[1])
    channel = scaled_data.columns.levels[1][0]
    for index, asset in enumerate(assets):
        if (index == 0):
            test_temp = test_data[asset, channel].values.reshape(-1,1)
            train_temp = train_data[asset, channel].values.reshape(-1,1)
            full_temp = scaled_data[asset, channel].values.reshape(-1,1)
        else:
            test_temp = np.concatenate((test_temp, test_data[asset, channel].values.reshape(-1,1)), axis=1)
            train_temp = np.concatenate((train_temp, train_data[asset, channel].values.reshape(-1,1)), axis=1)
            full_temp = np.concatenate((full_temp, scaled_data[asset, channel].values.reshape(-1,1)), axis=1)
            
    data_block_test = test_temp.reshape(n_channels,test_temp.shape[0],test_temp.shape[1])
    data_block_train = train_temp.reshape(n_channels,train_temp.shape[0],train_temp.shape[1])
    data_block = full_temp.reshape(n_channels,full_temp.shape[0],full_temp.shape[1])
    
    X_train, X_test, y_train, y_test, X_pred = [], [], [], [], []
    
    for i in range(n_train_sample):
        X_train.append(data_block_train[:,i:(i+LOOKBACK),:])
        y_train.append(data_block_train[:,(i+LOOKBACK):(i + LOOKBACK + HORIZON),:])
        
    for i in range(n_train_sample):
        X_test.append(data_block_test[:,i:(i+LOOKBACK),:])
        y_test.append(data_block_test[:,(i+LOOKBACK):(i + LOOKBACK + HORIZON),:])
        
    X_pred.append(data_block_test[:,-LOOKBACK:,:])
        
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), np.array(X_pred)

"""Here will start train and predict rolling window, getting weight for each rebalance date"""
portfolio_weights = []
for d in pred_dates:
    data = get_window(df, d)
    
    # split 100 sample for train, 100 sample for test. Since we have 190 samples, some test data will overlap
    X_train, y_train, X_test, y_test, X_pred = create_input(data)
    print(f"X_train shape = {X_train.shape}")
    print(f"y_train shape = {y_train.shape}")
    print(f"X_test shape = {X_test.shape}")
    print(f"y_test shape = {y_test.shape}")
    print(f"X_pred shape = {X_pred.shape}")
    
    model = build_model(
    n_assets=N_ASSETS,
    input_shape = (N_FEATURES),
    dropout=0.5
    )

    my_loss = make_my_loss([0.1, 0.2, 0.7])
    model.compile(
        loss= my_loss,
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    )
    model.summary()
    
    X_train_in = X_train.reshape(X_train.shape[0], -1)
    X_test_in = X_test.reshape(X_test.shape[0], -1)
    
    history = model.fit(
        X_train_in,
        y_train,
        validation_data=(X_test_in, y_test),
        epochs=N_EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es]
        )
    
    X_pred_in = X_pred.reshape(X_pred.shape[0], -1)
    weights = model.predict(X_pred_in)
    portfolio_weights.append(weights)

weights_df = pd.DataFrame(weights, columns=assets, index=pred_dates)

X_train shape = (40, 1, 40, 5)
y_train shape = (40, 1, 20, 5)
X_test shape = (40, 1, 40, 5)
y_test shape = (40, 1, 20, 5)
X_pred shape = (1, 1, 40, 5)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1005      
_________________________________________________________________
softmax (Softmax)            (None, 5)                 0         
Total params: 1,005
Trainable params: 1,005
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000


2021-10-01 15:53:59.691868: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-10-01 15:53:59.764013: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2/2 [==============================] - 3s 598ms/step - loss: -0.6080 - val_loss: -0.9911
Epoch 2/1000
2/2 [==============================] - 0s 70ms/step - loss: -0.6143 - val_loss: -0.9933
Epoch 3/1000
2/2 [==============================] - 0s 127ms/step - loss: -0.6011 - val_loss: -0.9955
Epoch 4/1000
2/2 [==============================] - 0s 120ms/step - loss: -0.5714 - val_loss: -0.9978
Epoch 5/1000
2/2 [==============================] - 0s 126ms/step - loss: -0.6165 - val_loss: -1.0000
Epoch 6/1000
2/2 [==============================] - 0s 154ms/step - loss: -0.5892 - val_loss: -1.0020
Epoch 7/1000
2/2 [==============================] - 0s 125ms/step - loss: -0.6028 - val_loss: -1.0042
Epoch 8/1000
2/2 [==============================] - 0s 132ms/step - loss: -0.6249 - val_loss: -1.0064
Epoch 9/1000
2/2 [==============================] - 0s 133ms/step - loss: -0.6208 - val_loss: -1.0087
Epoch 10/1000
2/2 [==============================] - 0s 179ms/step - loss: -0.6236 - val_loss: -

In [ ]:
fig = plt.figure(figsize=(8, 8), facecolor=background_color)
stack_list = []
for asset in assets:
    stack_list.append(weights_df[asset])
        
plt.stackplot(weights_df.index, stack_list)